To-do:
1. Текст был очищен только от одного мусорного элемента в качестве примера. Исслудйте данные через ноутбук или чере веб-интерфейс BigQuery на предмет других мусорных элементов в тексте, которые не несут в себе никакого особого смысла, а только создают шум в данных. Доработайте функцию очистки тектосвых данных, чтобы в нее можно было передать список ненужного мусора и разом выполнялась очистка
2. Проведите стратифицировнную кросс-валидуцию нейросетевого классификатора https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html
3. Поэксперементируйте с гиперпараметрами нейросетевого классификатора, постарайтесь повысить качество его работы
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
4. Попробуйте использовать не Word2Vec для получения векторого представления текста, а TF-IDF преобразование http://zabaykin.ru/?p=558 http://nlpx.net/archives/57
5. Попробуйте использовать более тонко настриваемые алгоритмы нейросетей, например из этого видео https://www.youtube.com/watch?v=cPkH1k3U1c8 

In [1]:
import warnings
warnings.filterwarnings('ignore')

from google.oauth2 import service_account
import pandas_gbq 

import numpy as np
import pandas as pd
import math as mt
import datetime as dt

from langdetect import detect
import re

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report,confusion_matrix
from gensim.models.word2vec import Word2Vec
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve,auc
import matplotlib.pyplot as plt

In [2]:
#import warnings
#warnings.filterwarnings('ignore')

#!pip install langdetect
#!pip install --user gensim
#!pip install gensim

In [3]:
CREDENTIALS = 

In [4]:
#funtion for getting fresh data from DWH for workload model
"""[summary]
Funtion for getting fresh data from BigQuery for workload scoring model
[description]
Credentials - google service account object with credentials data for project
[example]
Input: Credentials = credentials_object
Output: description	                                        channel	 category	category_flag
        \nChat transcript:\nVisitor: I want to buy wit...	chat	 ps	        1
        \nChat transcript:\nVisitor: hell i had a prob...	chat	 ps	        1
        \nChat transcript:\nVisitor: لا استطيع الشراء ...	 chat	  ps	     1
"""
def getDwhData(Credentials):
    statement_bigquery_sql = " ".join(["select description, channel, case",
                                       "when manual_category in ('payment_problem','how_to_pay','howtopay','how_to_play','paystation_error','ps_problem','ps_declined') then 'ps'",
                                       "else 'other'",
                                       "end as category,",
                                       "case",
                                       "when manual_category in ('payment_problem','how_to_pay','howtopay','how_to_play','paystation_error','ps_problem','ps_declined') then 0",
                                       "else 1",
                                       "end as category_flag",
                                       "from `xsolla_summer_school.customer_support`",
                                       "where manual_category is not null and",
                                       "manual_category <> '' and",
                                       "description is not null and",
                                       "description <> '' and",
                                       "channel is not null and",
                                       "channel <> '' and",
                                       "channel in ('chat','facebook')"])
    
    dataframe_bigquery = pandas_gbq.read_gbq(statement_bigquery_sql,project_id='findcsystem', credentials=Credentials, dialect='standard')

    return dataframe_bigquery


"""[summary]
Function for transform text to lower case
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["Text_1","Text_2"]
Output: ["text_1","text_2"]
"""
def lowerCase(Corpus):
    corpus = [i.lower().replace('\n','') for i in Corpus]
    return corpus


"""[summary]
Function for getting language of text
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["Text_1","Text_2"]
Output: ["en","ru"]
"""
def getTextLanguage(Corpus):
    txt_lang = []
    for txt in Corpus:
        try:
            lang = detect(txt)
            txt_lang.append(lang)
        except:
            lang = 'error'
            txt_lang.append(lang)
    
    return txt_lang


"""[summary]
Function for tokenization text
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["word1 word2","word3 word4"]
Output: [["word1","word2"],["word3","word4"]]
"""  
def textToTokens(Corpus):
    corpus = [i.split() for i in Corpus]
    return corpus 


"""[summary]
Function for clear text after garbage
[description]
Corpus - list or array object, with text data
Sub - list, regular expression
[example]
Input: Corpus = [["word1","word2"],["word3","word4"]]
       Substr = ["word1"]
Output: [["word2"],["word3","word4"]]
"""  
def clearTextAfterGarbage(Corpus,Sub):
    for k in Sub:
        Substr=k
        clear_corpus = []
        for text in Corpus:
            indexes = []
            text_len = len(text)
            try:
                for i in range(0,text_len):
                    res = re.search(Substr,text[i])
                    if res != None:
                        indexes.append(i)

                #delete garbage word from text
                for index in indexes:
                    del text[index]

                clear_corpus.append(text)
            except:
                clear_corpus.append("error")
        
    return clear_corpus


"""[summary]
Build word vector by using pre-trained Word2Vec model
[description]
Size - lenght of vector
Word2Vec_Model - gensim object
"""  
def buildWordVector(Text,Size,Word2Vec_Model):
    vec = np.zeros(Size).reshape((1,Size))
    count = 0.

    for word in Text:
        try:
            vec += Word2Vec_Model[word].reshape((1,Size))
            count += 1.
        except KeyError:
            continue
    
    if count != 0:
        vec /= count
    
    return vec

RAWDATA 

In [5]:
#getting data from dwh
SupportRawDataframe = getDwhData(CREDENTIALS)
SupportRawDataframe.shape

Downloading: 100%|███████████████████████████████████████████████████████████| 23450/23450 [00:11<00:00, 1959.50rows/s]


(23450, 4)

In [6]:
SupportRawDataframe.head(10)

,description,channel,category,category_flag
0,\nChat transcript:\nVisitor: I want to buy wit...,chat,ps,0
1,\nChat transcript:\nVisitor: hell i had a prob...,chat,ps,0
2,\nChat transcript:\nVisitor: لا استطيع الشراء ...,chat,ps,0
3,\nChat transcript:\nVisitor: im having trouble...,chat,ps,0
4,\nChat transcript:\nVisitor: Hi\nAna: Hello. H...,chat,ps,0
5,\nChat transcript:\nVisitor: Здраствуйте\nAna:...,chat,ps,0
6,\nChat transcript:\nVisitor: hello\nVisitor: w...,chat,ps,0
7,\nChat transcript:\nVisitor: not letting me bu...,chat,ps,0
8,\nChat transcript:\nVisitor: مرحبا\nAna: Hello...,chat,ps,0
9,\nChat transcript:\nVisitor: hi im trying to b...,chat,ps,0


In [7]:
SupportRawDataframe.tail(10)

,description,channel,category,category_flag
23440,"\nChat transcript:\nVisitor: Hello, I bought p...",chat,other,1
23441,\nChat transcript:\nVisitor: Good day\n\nI acc...,facebook,other,1
23442,\nChat transcript:\nVisitor: payment not going...,chat,other,1
23443,\nChat transcript:\nVisitor: Paid for faceit s...,chat,other,1
23444,"\nChat transcript:\nVisitor: Hello, I am tryin...",chat,other,1
23445,"\nChat transcript:\nVisitor: Hi, i made a pruc...",chat,other,1
23446,"\nChat transcript:\nVisitor: Hi, how long will...",chat,other,1
23447,\nChat transcript:\nVisitor: I bought playerun...,chat,other,1
23448,\nChat transcript:\nVisitor: Good day i took t...,chat,other,1
23449,\nChat transcript:\nVisitor: hi\nVisitor: hell...,chat,other,1


DATA PREPROC

In [8]:
#transform text to lower case
corpus = SupportRawDataframe.description
corpus.astype('str')

corpus = lowerCase(corpus)

#getting language for text corpus
corpus_lang = getTextLanguage(corpus)

In [9]:
#new dataframe with texts in lower case, without /n symbol and with lang for text
SupportRawDataframe['description'] = corpus
SupportRawDataframe['lang'] = corpus_lang

In [10]:
SupportRawDataframe.head(10)

,description,channel,category,category_flag,lang
0,chat transcript:visitor: i want to buy with pa...,chat,ps,0,en
1,chat transcript:visitor: hell i had a problem ...,chat,ps,0,en
2,chat transcript:visitor: لا استطيع الشراء ومعل...,chat,ps,0,en
3,chat transcript:visitor: im having trouble wit...,chat,ps,0,en
4,chat transcript:visitor: hiana: hello. how can...,chat,ps,0,en
5,chat transcript:visitor: здраствуйтеana: здрав...,chat,ps,0,ru
6,chat transcript:visitor: hellovisitor: why i c...,chat,ps,0,en
7,chat transcript:visitor: not letting me buy an...,chat,ps,0,en
8,chat transcript:visitor: مرحباana: hello!visit...,chat,ps,0,en
9,chat transcript:visitor: hi im trying to buy v...,chat,ps,0,en


In [11]:
SupportRawDataframe.tail(10)

,description,channel,category,category_flag,lang
23440,"chat transcript:visitor: hello, i bought pubg ...",chat,other,1,en
23441,chat transcript:visitor: good dayi accidentall...,facebook,other,1,en
23442,chat transcript:visitor: payment not going thr...,chat,other,1,en
23443,chat transcript:visitor: paid for faceit subsc...,chat,other,1,en
23444,"chat transcript:visitor: hello, i am trying to...",chat,other,1,en
23445,"chat transcript:visitor: hi, i made a pruchase...",chat,other,1,en
23446,"chat transcript:visitor: hi, how long will it ...",chat,other,1,en
23447,chat transcript:visitor: i bought playerunknow...,chat,other,1,en
23448,chat transcript:visitor: good day i took the w...,chat,other,1,en
23449,chat transcript:visitor: hivisitor: hellovisit...,chat,other,1,en


In [12]:
#getting only en texts
SupportDataframe_eng = SupportRawDataframe[SupportRawDataframe.lang == 'en'][:]
SupportDataframe_eng

,description,channel,category,category_flag,lang
0,chat transcript:visitor: i want to buy with pa...,chat,ps,0,en
1,chat transcript:visitor: hell i had a problem ...,chat,ps,0,en
2,chat transcript:visitor: لا استطيع الشراء ومعل...,chat,ps,0,en
3,chat transcript:visitor: im having trouble wit...,chat,ps,0,en
4,chat transcript:visitor: hiana: hello. how can...,chat,ps,0,en
...,...,...,...,...,...
23445,"chat transcript:visitor: hi, i made a pruchase...",chat,other,1,en
23446,"chat transcript:visitor: hi, how long will it ...",chat,other,1,en
23447,chat transcript:visitor: i bought playerunknow...,chat,other,1,en
23448,chat transcript:visitor: good day i took the w...,chat,other,1,en


In [13]:
#text tekenization
tokenization = textToTokens(SupportDataframe_eng.description)
SupportDataframe_eng['description'] = tokenization

In [14]:
SupportDataframe_eng

,description,channel,category,category_flag,lang
0,"[chat, transcript:visitor:, i, want, to, buy, ...",chat,ps,0,en
1,"[chat, transcript:visitor:, hell, i, had, a, p...",chat,ps,0,en
2,"[chat, transcript:visitor:, لا, استطيع, الشراء...",chat,ps,0,en
3,"[chat, transcript:visitor:, im, having, troubl...",chat,ps,0,en
4,"[chat, transcript:visitor:, hiana:, hello., ho...",chat,ps,0,en
...,...,...,...,...,...
23445,"[chat, transcript:visitor:, hi,, i, made, a, p...",chat,other,1,en
23446,"[chat, transcript:visitor:, hi,, how, long, wi...",chat,other,1,en
23447,"[chat, transcript:visitor:, i, bought, playeru...",chat,other,1,en
23448,"[chat, transcript:visitor:, good, day, i, took...",chat,other,1,en


In [15]:
lst=['chat','hi','hellow','transcript:visitor:','hiana:','anna','thank you','please','wait','?','!','ок','окy','u','May','Joel','thankx','we', 'you','he','she','it','they','me','us','him','them','mine','yours','her','hers','his','its','our', 'ours','their','theirs','the','to','with','of','on','in','under','above','as','after','at','before','by','will','but','what','which','who','for','from','out']

texts = list(SupportDataframe_eng.description)
tests_clear =clearTextAfterGarbage(texts,lst)

In [16]:
SupportDataframe_eng['description'] = tests_clear
SupportDataframe_eng

,description,channel,category,category_flag,lang
0,"[i, want, paypal, and, show, a, problem, call,...",chat,ps,0,en
1,"[i, had, a, problem, yesterday, people, i, a, ...",chat,ps,0,en
2,"[لا, استطيع, الشراء, ومعلومات, الشراء, صحيحة, ...",chat,ps,0,en
3,"[im, new, xsolla, auth, digit, making, pymaent...",chat,ps,0,en
4,"[how, can, i, help, can't, pay, paypal, have, ...",chat,ps,0,en
...,...,...,...,...,...
23445,"[i, made, a, $25, dollars, and, send, my, wall...",chat,other,1,en
23446,"[how, take, my, friend, receive, pack?michael:...",chat,other,1,en
23447,"[i, playerunknowns, and, like, a, refundalexan...",chat,other,1,en
23448,"[good, day, i, package, i, wanted, to, and, to...",chat,other,1,en


TRAIN TEST SPLIT DATAFRAMES

In [17]:
#list of unique categories
unique_categories = np.unique(SupportDataframe_eng.category)
unique_categories

array(['other', 'ps'], dtype=object)

In [18]:
descriptions = SupportDataframe_eng['description']
categories = SupportDataframe_eng['category_flag']
XTrain,XTest,YTrain,YTest = train_test_split(descriptions,
                                             categories,
                                             stratify = categories,
                                             test_size = 0.2,
                                             random_state = 40)

FROM TEXTS TO VECTORS

In [19]:
#initialize Word2Vec model for embedding words to vectors
NDim = 100
Imdb_w2v = Word2Vec(size = NDim,min_count = 10)
Imdb_w2v.build_vocab(XTrain)

In [20]:
Imdb_w2v.train(XTrain,total_examples = Imdb_w2v.corpus_count,epochs = Imdb_w2v.epochs)

(3031967, 4650880)

In [21]:
#embedding training messages to vectors for neutral classifier
TrainVecs = np.concatenate([buildWordVector(i,NDim,Imdb_w2v) for i in XTrain])

In [22]:
Imdb_w2v.train(XTest, total_examples = Imdb_w2v.corpus_count, epochs = Imdb_w2v.epochs)

(764863, 1181520)

In [23]:
TestVecs = np.concatenate([buildWordVector(i,NDim,Imdb_w2v) for i in XTest])

CROSSVALIDATION AND BUILD CLASSIFIER

In [24]:
TextClassifier = MLPClassifier(hidden_layer_sizes = (20,10), max_iter = 1000, random_state = 40)

In [25]:
from sklearn.model_selection import StratifiedKFold
cv=StratifiedKFold(n_splits=5)
Scores = cross_val_score(TextClassifier, TrainVecs, YTrain, cv = 5)
print(Scores)
print(np.mean(Scores))

[0.8836392  0.87856306 0.88129637 0.88715346 0.87578125]
0.8812866678055447


In [26]:
TextClassifier.fit(TrainVecs,YTrain)

MLPClassifier(hidden_layer_sizes=(20, 10), max_iter=1000, random_state=40)

In [27]:
pred = TextClassifier.predict(TestVecs)
print(confusion_matrix(YTest,pred))
print(classification_report(YTest,pred))

[[ 321  316]
 [  92 2472]]
              precision    recall  f1-score   support

           0       0.78      0.50      0.61       637
           1       0.89      0.96      0.92      2564

    accuracy                           0.87      3201
   macro avg       0.83      0.73      0.77      3201
weighted avg       0.86      0.87      0.86      3201



Случайный андерсэмплинг и оверсэмплинг

*так себе результат

In [28]:
X = pd.DataFrame(TrainVecs)
X['target'] = YTrain

num_0 = len(X[X['target']==0])
num_1 = len(X[X['target']==1])

undersampled_data_train = pd.concat([X[X['target']==0].sample(num_1, replace=True), X[X['target']==1] ])

oversampled_data_train = pd.concat([ X[X['target']==0] , X[X['target']==1].sample(num_0, replace=True) ])

In [29]:
# случайный андерсэмплинг
undersampled_data_train = pd.concat([X[X['target']==0].sample(num_1, replace=True), X[X['target']==1] ])
yy=undersampled_data_train['target']
del undersampled_data_train['target']
TextClassifier.fit(undersampled_data_train,yy)
pred = TextClassifier.predict(TestVecs)
print(confusion_matrix(YTest,pred))
print(classification_report(YTest,pred))

[[ 250  386]
 [ 825 1738]]
              precision    recall  f1-score   support

           0       0.23      0.39      0.29       636
           1       0.82      0.68      0.74      2563

    accuracy                           0.62      3199
   macro avg       0.53      0.54      0.52      3199
weighted avg       0.70      0.62      0.65      3199



In [30]:
# случайный оверсэмплинг
oversampled_data_train
yyy=oversampled_data_train['target']
del oversampled_data_train['target']
TextClassifier.fit(oversampled_data_train,yyy)
pred = TextClassifier.predict(TestVecs)
print(confusion_matrix(YTest,pred))
print(classification_report(YTest,pred))

[[ 300  336]
 [1024 1539]]
              precision    recall  f1-score   support

           0       0.23      0.47      0.31       636
           1       0.82      0.60      0.69      2563

    accuracy                           0.57      3199
   macro avg       0.52      0.54      0.50      3199
weighted avg       0.70      0.57      0.62      3199



Оверсэмплинг со SMOTE

*лучший результат

In [39]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2,)

X_train_res, y_train_res = sm.fit_sample(TrainVecs, YTrain.ravel())

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1)))

print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0)))

After OverSampling, counts of label '1': 10251
After OverSampling, counts of label '0': 10251


In [40]:
TextClassifier = MLPClassifier(hidden_layer_sizes = (20,10), max_iter = 1000, random_state = 40)
cv=StratifiedKFold(n_splits=5)
Scores = cross_val_score(TextClassifier, TrainVecs, YTrain, cv = 5)
print(Scores)
print(np.mean(Scores))

[0.85892927 0.88042204 0.86869871 0.86283705 0.85301016]
0.8647794458901282


In [41]:
TextClassifier.fit(X_train_res,y_train_res)

MLPClassifier(hidden_layer_sizes=(20, 10), max_iter=1000, random_state=40)

In [42]:
X_test_res, y_test_res = sm.fit_sample(TestVecs, YTest.ravel())

In [43]:
pred = TextClassifier.predict(X_test_res)
print(confusion_matrix(y_test_res,pred))
print(classification_report(y_test_res,pred))

[[1777  786]
 [ 388 2175]]
              precision    recall  f1-score   support

           0       0.82      0.69      0.75      2563
           1       0.73      0.85      0.79      2563

    accuracy                           0.77      5126
   macro avg       0.78      0.77      0.77      5126
weighted avg       0.78      0.77      0.77      5126

